# MVA - Homework 1 - Reinforcement Learning (2021/2022)



## Instructions

* The deadline is **November 12 at 11:59 pm (Paris time).**

* By doing this homework you agree to the late day policy, collaboration and misconduct rules reported on [Piazza](https://piazza.com/class/ktmvsc4knke4ia?cid=6).

* **Mysterious or unsupported answers will not receive full credit**. A correct answer, unsupported by calculations, explanation, or algebraic work will receive no credit; an incorrect answer supported by substantially correct calculations and explanations might still receive partial credit.

* Answers should be provided in **English**.

# Colab setup

In [ ]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
  # install rlberry library
  !pip install git+https://github.com/rlberry-py/rlberry.git@mva2021#egg=rlberry[default] > /dev/null 2>&1

  # install ffmpeg-python for saving videos
  !pip install ffmpeg-python > /dev/null 2>&1

  # packages required to show video
  !pip install pyvirtualdisplay > /dev/null 2>&1
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

  print("Libraries installed, please restart the runtime!")


Libraries installed, please restart the runtime!


In [ ]:
# Create directory for saving videos
!mkdir videos > /dev/null 2>&1

# Initialize display and import function to show videos
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

In [ ]:
# Useful libraries
import numpy as np
import matplotlib.pyplot as plt

# Preparation

In the coding exercises, you will use a *grid-world* MDP, which is represented in Python using the interface provided by the [Gym](https://gym.openai.com/) library. The cells below show how to interact with this MDP and how to visualize it.


In [ ]:
from rlberry.envs import GridWorld

def get_env():
  """Creates an instance of a grid-world MDP."""
  env = GridWorld(
      nrows=5,
      ncols=7,
      reward_at = {(0, 6):1.0},
      walls=((0, 4), (1, 4), (2, 4), (3, 4)),
      success_probability=0.9,
      terminal_states=((0, 6),)
  )
  return env

def render_policy(env, policy=None, horizon=50):
  """Visualize a policy in an environment

  Args:
    env: GridWorld
        environment where to run the policy
    policy: np.array
        matrix mapping states to action (Ns).
        If None, runs random policy.
    horizon: int
        maximum number of timesteps in the environment.
  """
  env.enable_rendering()
  state = env.reset()                       # get initial state
  for timestep in range(horizon):
      if policy is None:
        action = env.action_space.sample()  # take random actions
      else:
        action = policy[state]
      next_state, reward, is_terminal, info = env.step(action)
      state = next_state
      if is_terminal:
        break
  # save video and clear buffer
  env.save_video('./videos/gw.mp4', framerate=5)
  env.clear_render_buffer()
  env.disable_rendering()
  # show video
  show_video('./videos/gw.mp4')


In [ ]:
# Create an environment and visualize it
env = get_env()
render_policy(env)  # visualize random policy

# The reward function and transition probabilities can be accessed through
# the R and P attributes:
print(f"Shape of the reward array = (S, A) = {env.R.shape}")
print(f"Shape of the transition array = (S, A, S) = {env.P.shape}")
print(f"Reward at (s, a) = (1, 0): {env.R[1, 0]}")
print(f"Prob[s\'=2 | s=1, a=0]: {env.P[1, 0, 2]}")
print(f"Number of states and actions: {env.Ns}, {env.Na}")

# The states in the griworld correspond to (row, col) coordinates.
# The environment provides a mapping between (row, col) and the index of
# each state:
print(f"Index of state (1, 0): {env.coord2index[(1, 0)]}")
print(f"Coordinates of state 5: {env.index2coord[5]}")

# Part 1 - Dynamic Programming

## Question 1.1

Consider a general MDP with a discount factor of $\gamma < 1$. Assume that the horizon is infinite (so there is no termination). A policy $\pi$ in this MDP
induces a value function $V^\pi$. Suppose an affine transformation is applied to the reward, what is
the new value function? Is the optimal policy preserved?



### **Answer**

Let $ a, b $ be two real scalars. Consider the reward transformation $R' = ar + b$, where $r$ is the original reward, and $R$ is the new reward after the affine transformation. Let $V^{\pi}(s)$ be the value function under policy $ \pi$ with the original reward $r$, and let $ \bar{V}^{\pi}(s)$ be the value function under the same policy $\pi$ with the transformed reward $R'$.

We have:

$$
\bar{V}^{\pi}(s) = \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t R'(s_t, a_t) \mid s_0 = s, \pi \right].
$$

Substituting $ R'(s_t, a_t) = ar(s_t, a_t) + b $, we get:

$$
\bar{V}^{\pi}(s) = \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t (ar(s_t, a_t) + b) \mid s_0 = s, \pi \right].
$$

This can be rewritten as:

$$
\bar{V}^{\pi}(s) = \mathbb{E}\left[a \sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) + b \sum_{t=0}^{\infty} \gamma^t \mid s_0 = s, \pi \right].
$$

Using the linearity of expectation:

$$
\bar{V}^{\pi}(s) = a \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) \mid s_0 = s, \pi \right] + b \sum_{t=0}^{\infty} \gamma^t.
$$

The first term is just $ aV^{\pi}(s)$, and the second term is a geometric series that sums to $ \frac{1}{1-\gamma}$. Therefore:

$$
\bar{V}^{\pi}(s) = aV^{\pi}(s) + \frac{b}{1-\gamma}.
$$

- **Is the optimal policy preserved?**

The optimal policy will only be preserved if $ a > 0 $. This is because:

$$
\text{Argmax}_{\pi} \bar{V}^{\pi}(s) = \text{Argmax}_{\pi} \left(a V^{\pi}(s) + \frac{b}{1-\gamma}\right) = \text{Argmax}_{\pi} V^{\pi}(s).
$$

Since $ \frac{b}{1-\gamma}$ is a constant independent of the policy $ \pi$, it does not affect the argmax operation. The scaling factor $ a$ must be positive to ensure that the ordering of policies (in terms of their value functions) is preserved. If $ a < 0 $, the optimal policy would be reversed, which would not preserve the original optimal policy.


## Question 1.2

Consider an infinite-horizon $\gamma$-discounted MDP. We denote by $Q^*$ the $Q$-function of the optimal policy $\pi^*$. Prove that, for any function $Q(s, a)$ (which is **not** necessarily the value function of a policy), the following inequality holds for any state $s$:

$$
V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma}||Q^*-Q||_\infty,
$$

where $||Q^*-Q||_\infty = \max_{s, a} |Q^*(s, a) - Q(s, a)|$ and $\pi_Q(s) \in \arg\max_a Q(s, a)$. Can you use this result to show that any policy $\pi$ such that $\pi(s) \in \arg\max_a Q^*(s, a)$ is optimal?

### **Answer**


We begin by recalling that the value function $V^{\pi_Q}(s)$ of the policy $\pi_Q$ is given by:

$$
V^{\pi_Q}(s) = \mathbb{E} \left[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \mid s_0 = s, \pi_Q \right].
$$

Since $\pi_Q(s) = \arg\max_a Q(s, a)$, we can express $V^{\pi_Q}(s)$ as:

$$
V^{\pi_Q}(s) = Q(s, \pi_Q(s)) = \max_a Q(s, a).
$$

For the optimal policy $\pi^*$, the value function $V^*(s)$ is:

$$
V^*(s) = \max_a Q^*(s, a).
$$

Now, let's examine the difference between $V^{\pi_Q}(s)$ and $V^*(s)$:

$$
V^{\pi_Q}(s) - V^*(s) = \max_a Q(s, a) - \max_a Q^*(s, a).
$$

Using the property of norms, we know that for any functions $f(a)$ and $g(a)$:

$$
\max_a f(a) - \max_a g(a) \geq -\max_a |f(a) - g(a)|.
$$

Applying this to our case:

$$
V^{\pi_Q}(s) - V^*(s) \geq -\max_a |Q(s, a) - Q^*(s, a)|.
$$

Since the above inequality holds for any state $s$, we can write:

$$
V^{\pi_Q}(s) \geq V^*(s) - \max_a |Q(s, a) - Q^*(s, a)|.
$$

By the definition of the $L_\infty$ norm:

$$
||Q^* - Q||_\infty = \max_{s, a} |Q^*(s, a) - Q(s, a)|.
$$

Thus, we have:

$$
V^{\pi_Q}(s) \geq V^*(s) - ||Q^* - Q||_\infty.
$$

Next, we use the fact that the Bellman operator $T$ for $V^*$ satisfies:

$$
V^*(s) = \max_a \mathbb{E} \left[ r(s, a) + \gamma V^*(s') \right].
$$

The contraction property of the Bellman operator gives:

$$
|V^{\pi_Q}(s) - V^*(s)| \leq \frac{\gamma}{1-\gamma} ||Q^* - Q||_\infty.
$$

Adding the two inequalities:

$$
V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma} ||Q^* - Q||_\infty.
$$

This proves the inequality stated in the problem.  \\

Finally, if $\pi(s) \in \arg\max_a Q^*(s, a)$, then $Q(s, \pi(s)) = \max_a Q^*(s, a)$, and $V^{\pi}(s) = V^*(s)$. Therefore, $\pi$ is an optimal policy.



## Question 1.3

In this question, you will implement and compare the policy and value iteration algorithms for a finite MDP.

Complete the functions `policy_evaluation`, `policy_iteration` and `value_iteration` below.


Compare value iteration and policy iteration. Highlight pros and cons of each method.

### **Answer**

Pros and cons of each method:

**Value iteration:**

* Simpler to implement and more intuitive algorithm
* Computationally expensive
* Slow convergence.

**Value inversion:**

* fastest method
* Almost exact result in simple games
* WOuld be much slower in high dimension since the complexity of matrix inversion is $O(d^3)$

**Policy iteration:**

* more complex algorithm
* faster than value iteration
* fast convergence





In [ ]:
from numpy.linalg import inv
def policy_evaluation(P, R, policy, gamma=0.9, tol=1e-2):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        policy: np.array
            matrix mapping states to action (Ns)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        value_function: np.array
            The value function of the given policy
    """
    Ns, Na = R.shape
    value_function = np.zeros(Ns)
    Reward_policy=np.zeros((Ns,))
    Matrix_policy=np.zeros((Ns,Ns))
    for i in range(Ns):
      Reward_policy[i]=R[i,policy[i]]
      for j in range(Ns):
        Matrix_policy[i,j]=P[i,policy[i],j]
    value_function=np.linalg.solve(np.eye(Ns) - gamma * Matrix_policy, Reward_policy)
    #value_function= inv(np.eye(Ns) - gamma * Matrix_policy).dot(Reward_policy)
    return value_function

In [ ]:
def policy_iteration(P, R, gamma=0.9, tol=1e-20):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        policy: np.array
            the final policy
        V: np.array
            the value function associated to the final policy
    """
    Ns, Na = R.shape
    V = np.zeros(Ns)
    policy = np.zeros(Ns).astype(int)
    it=0
    while np.max(abs(policy_evaluation(P, R, policy,tol)-V))!=0 :

      V=np.copy(policy_evaluation(P, R, policy,tol))
      for i in range(Ns):
        liste=[]
        for a in range(Na):
          n=0
          for j in range(Ns):
            n+=policy_evaluation(P, R, policy,tol)[j]*P[i,a,j]
            #print(n)
          liste=liste+[R[i,a]+n*gamma]
        policy[i]=l.index(max(l))
        it+=1
      #print("new policy", policy)
      #print("new error", np.max(abs(policy_evaluation(P, R, policy,tol)-V)))
    return policy, V

In [ ]:
def value_iteration(P, R, gamma=0.9, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        Q: final Q-function (at iteration n)
        greedy_policy: greedy policy wrt Qn
        Qfs: all Q-functions generated by the algorithm (for visualization)
    """
    Q = np.zeros_like(R)
    Ns, Na = R.shape

    Qfs = []
    B=np.copy(Q) + 1
    while np.max(abs(Q-B))>tol:
      B=np.copy(Q)
      Qfs+=[B]

      for i in range(Ns):
        for a in range(Na):
          s=0
          for j in range(Ns):
            s+=max(Q[j,])*P[i,a,j]
          Q[i,a]=R[i,a]+s*gamma

    greedy_policy = np.argmax(Q, axis=1)

    return Q, greedy_policy, Qfs

### Testing your code

In [ ]:
# Parameters
tol = 1e-5
gamma = 0.99

# Environment
env = get_env()

# run value iteration to obtain Q-values
VI_Q, VI_greedypol, all_qfunctions =  value_iteration(env.P, env.R, gamma=gamma, tol=tol)
# render the policy
print("[VI]Greedy policy: ")
render_policy(env, VI_greedypol)

# compute the value function of the greedy policy using matrix inversion
from numpy.linalg import inv
Ns=31
Matrix_policy=np.zeros((Ns,Ns))
for i in range(Ns):
  for j in range(Ns):
    Matrix_policy[i,j]=env.P[i,VI_greedypol[i],j]
Reward_policy=np.zeros((Ns,))
for i in range(Ns):
  Reward_policy[i]=env.R[i,VI_greedypol[i]]
Value_greedy_policy=inv(np.identity(Ns)-gamma*Matrix_policy)@ Reward_policy


# compute value function of the greedy policy
#

# ====================================================

# show the error between the computed V-functions and the final V-function
# (that should be the optimal one, if correctly implemented)
# as a function of time
final_V = all_qfunctions[-1].max(axis=1)
norms = [ np.linalg.norm(q.max(axis=1) - final_V) for q in all_qfunctions]
plt.plot(norms)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Value iteration: convergence")

#### POLICY ITERATION ####
PI_policy, PI_V = policy_iteration(env.P, env.R, gamma=gamma, tol=tol)
print("\n[PI]final policy: ")
render_policy(env, PI_policy)

# Uncomment below to check that everything is correct
assert np.allclose(PI_policy, VI_greedypol),\
    "You should check the code, the greedy policy computed by VI is not equal to the solution of PI"
assert np.allclose(PI_V, greedy_V),\
    "Since the policies are equal, even the value function should be"

plt.show()

# Part 2 - Tabular RL

## Question 2.1

The code below collects two datasets of transitions (containing states, actions, rewards and next states) for a discrete MDP.

For each of the datasets:

1. Estimate the transitions and rewards, $\hat{P}$ and $\hat{R}$.
2. Compute the optimal value function and the optimal policy with respect to the estimated MDP (defined by $\hat{P}$ and $\hat{R}$), which we denote by $\hat{\pi}$ and $\hat{V}$.
3. Numerically compare the performance of $\hat{\pi}$ and $\pi^\star$ (the true optimal policy), and the error between $\hat{V}$ and $V^*$ (the true optimal value function).

Which of the two data collection methods do you think is better? Why?

### **Answer**

The second data collection , because if the the random policy chosen to collect the first data collection is not good " or optimal", we will get very bad samples. On the other hand, it's less risky to collect samples for the the second data collection with a uniform policy.

In [ ]:
def get_random_policy_dataset(env, n_samples):
  """Get a dataset following a random policy to collect data."""
  states = []
  actions = []
  rewards = []
  next_states = []

  state = env.reset()
  for _ in range(n_samples):
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.step(action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    # update state
    state = next_state
    if is_terminal:
      state = env.reset()

  dataset = (states, actions, rewards, next_states)
  return dataset

def get_uniform_dataset(env, n_samples):
  """Get a dataset by uniformly sampling states and actions."""
  states = []
  actions = []
  rewards = []
  next_states = []
  for _ in range(n_samples):
    state = env.observation_space.sample()
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.sample(state, action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)

  dataset = (states, actions, rewards, next_states)
  return dataset


# Collect two different datasets
num_samples = 500
env = get_env()
dataset_1 = get_random_policy_dataset(env, num_samples)
dataset_2 = get_uniform_dataset(env, num_samples)


# Item 3: Estimate the MDP with the two datasets; compare the optimal value
# functions in the true and in the estimated MDPs

#Dataset1:
liste1=[]
for i in range(len(dataset_1[0])):
  liste1+=[(dataset_1[0][i],dataset_1[1][i],dataset_1[2][i],dataset_1[3][i])]
  liste1_R=[(k[0],k[1]) for k in liste1]      #reward matrix

#Reward matrix for the first dataset
R1=np.zeros((env.Ns,env.Na))
for i in range(len(dataset_1[0])):
  R1[liste1[i][0],liste1[i][1]]+=liste1[i][2]
for i in range(31):
  for a in range(4):
    if liste1_R.count((i,a))!=0:
      R1[i,a]=R1[i,a]/liste1_R.count((i,a))
    else:
      R1[i,a]=0
# Transisition matrix
P1=np.zeros((env.Ns,env.Na, env.Ns))
liste1_P=[(K[0],K[1],K[3]) for K in liste1]
for i in liste1_P:
  P1[i[0],i[1],i[2]]+=1/liste1_R.count((i[0],i[1]))

# Dataset2:

liste2=[]
for i in range(len(dataset_2[0])):
  liste2+=[(dataset_2[0][i],dataset_2[1][i],dataset_2[2][i],dataset_2[3][i])]
liste2_R=[(k[0],k[1]) for k in liste2]
R2=np.zeros((31,4))
for i in range(len(dataset_2[0])):
  R2[liste2[i][0],liste2[i][1]]+=liste2[i][2]
for i in range(env.Ns):
  for a in range(env.Na):
    if liste2_R.count((i,a))!=0:
      R2[i,a]=R2[i,a]/liste2_R.count((i,a))
    else:
      R2[i,a]=0


#Transition matrix P2
P2=np.zeros((env.Ns,env.Na,env.Ns))
liste2_P=[(K[0],K[1],K[3]) for K in liste2]
for i in liste2_P:
  P2[i[0],i[1],i[2]]+=1/liste2_R.count((i[0],i[1]))


question : Optimal Policy

Let's consider a different policy_iteration fuction, in a way that if speed= False, it will give the policy and the value function.

if speed =True, it will return the policy, value function and the number of iterations.

In [ ]:
def policy_iteration2(P, R, gamma=0.9, tol=1e-10,speed=False):#when it is True return also the number of iteration
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        policy: np.array
            the final policy
        V: np.array
            the value function associated to the final policy
    """
    Ns, Na = R.shape
    V = np.zeros(Ns)
    policy = np.ones(Ns, dtype=np.int)
    it=0
    while np.max(abs(policy_evaluation(P, R, policy,tol)-V))!=0 :
      it+=1

      V=np.copy(policy_evaluation(P, R, policy,tol))
      for i in range(Ns):
        l=[]
        for a in range(Na):
          n=0
          for j in range(Ns):
            n+=policy_evaluation(P, R, policy,tol)[j]*P[i,a,j]

          l=l+[R[i,a]+n*gamma]
        policy[i]=l.index(max(l))
        it+=1

    return  policy, V



In [ ]:
## MDP1
optimal_policy1,value_function_1,nombre_d_iteration1  = policy_iteration2(P1, R2,speed=True)


print("optimal_policy1", optimal_policy1)
print("value_function_1", value_function_1)
print("nombre_d_iteration1", nombre_d_iteration1)

In [ ]:
##MDP2
optimal_policy2,value_function_2,nombre_d_iteration2 = policy_iteration2(P2, R2,speed=True)

print("optimal_policy2", optimal_policy2)
print("value_function_2", value_function_2)
print("nombre_d_iteration2", nombre_d_iteration2)

In [ ]:

True_optimale_policy,V_optimale_policy,nombre_d_iteration = policy_iteration2(env.P,env.R,speed=True)


True_optimale_policy,V_optimale_policy,nombre_d_iteration

## Question 2.2

Suppose that $\hat{P}$ and $\hat{R}$ are estimated from a dataset of exactly $N$ i.i.d. samples from **each** state-action pair. This means that, for each $(s,a)$, we have $N$ samples $\{(s_1',r_1, \dots, s_N', r_N\}$, where $s_i' \sim P(\cdot | s,a)$ and $r_i \sim R(s,a)$ for $i=1,\dots,N$, and
$$ \hat{P}(s'|s,a) = \frac{1}{N}\sum_{i=1}^N \mathbb{1}(s_i' = s'), $$
$$ \hat{R}(s,a) = \frac{1}{N}\sum_{i=1}^N r_i.$$
Suppose that $R$ is a distribution with support in $[0,1]$. Let $\hat{V}$ be the optimal value function computed in the empirical MDP (i.e., the one with transitions $\hat{P}$ and rewards $\hat{R}$). For any $\delta\in(0,1)$, derive an upper bound to the error

$$ \| \hat{V} - V^* \|_\infty $$

which holds with probability at least $1-\delta$.

**Note** Your bound should only depend on deterministic quantities like $N$, $\gamma$, $\delta$, $S$, $A$. It should *not* dependent on the actual random samples.

**Hint** The following two inequalities may be helpful.

1. **A (simplified) simulation lemma**. For any state $\bar{s}$,

$$ |\hat{V}(\bar{s}) - V^*(\bar{s})| \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$$

2. **Hoeffding's inequality**. Let $X_1, \dots X_N$ be $N$ i.i.d. random variables bounded in the interval $[0,b]$ for some $b>0$. Let $\bar{X} = \frac{1}{N}\sum_{i=1}^N X_i$ be the empirical mean. Then, for any $\epsilon > 0$,

$$ \mathbb{P}(|\bar{X} - \mathbb{E}[\bar{X}]| > \epsilon) \leq 2e^{-\frac{2N\epsilon^2}{b^2}}.$$

### **Answer**

\begin{align*}
\mathbb{P}(\|V^* - \hat{V}\|_\infty > \epsilon) &\leq \sum_{\bar{s} \in S} \mathbb{P}(|V^*(\bar{s})-\hat{V}(\bar{s})|> \epsilon)\\
&\leq \sum_{\bar{s} \in S} \mathbb{P}\left(\frac{1}{1-\gamma} \max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s' \in S} (P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| > \epsilon\right) \\
&\leq \sum_{\bar{s} \in S} \mathbb{P}\left(\frac{1}{1-\gamma} \max_{s,a} \left| R(s,a) + \gamma \sum_{s'\in S} P(s'|s,a) V^*(s') - \left(\hat{R}(s,a) + \gamma \sum_{s'\in S} \hat{P}(s'|s,a) V^*(s')\right)\right| > \epsilon\right) \\
&\leq \sum_{\bar{s} \in S} \sum_{s,a} \mathbb{P}\left(\frac{1}{1-\gamma} \left| Y - \mathbb{E}[Y] \right| > \epsilon\right),
\end{align*}

where $ Y = \hat{R}(s,a) + \gamma \sum_{s'\in S} \hat{P}(s'|s,a) V^*(s') = \frac{1}{N} \sum_{i=1}^N \left(r_i + \gamma \sum_{s' \in S} \mathbb{1}_{\{s_i'=s'\}} V^*(s')\right) = \frac{1}{N} \sum_{i=1}^N Y_i. $

$ Y_i$ are i.i.d. random variables.

- **Now let's show that $ Y_i \in [0, 1 + \gamma\|V^*\|_{\infty}] $.**

Since $ R $ is bounded, we assume its upper bound is 1. We have, for any policy $ \pi $,

$$V^{\pi}(s) = \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t R(s_t, a_t) \mid s_0 = s \right] \leq \sum_{t=0}^{\infty} \gamma^t = \frac{1}{1-\gamma}.
$$

Therefore, $ Y_i \in [0, \frac{1}{1-\gamma}] $. Applying Hoeffding's inequality to $ Y_i $:

$$
\mathbb{P}(|Y - \mathbb{E}[Y]| > \epsilon) \leq 2e^{-\frac{2N\epsilon^2}{b^2}},
$$

with $ N = N $, $ b = \frac{1}{1-\gamma} $.

We get:

$$
\mathbb{P}\left(\frac{1}{1-\gamma} |Y - \mathbb{E}[Y]| > \epsilon\right) = \mathbb{P}\left(|Y - \mathbb{E}[Y]| > \epsilon(1-\gamma)\right) \leq 2e^{-2\epsilon^2 N (1-\gamma)^2}.
$$

Therefore,

\begin{align*}
\mathbb{P}(\|V^* - \hat{V}\|_\infty > \epsilon) &\leq \sum_{s \in S} \sum_{s,a} 2e^{-2\epsilon^2 N (1-\gamma)^2} \\
&\leq 2 |A||S|^2 e^{-2\epsilon^2 N (1-\gamma)^2}.
\end{align*}

Taking $ \epsilon' = \frac{1}{\sqrt{2N}} \times \frac{1}{1-\gamma} \times \sqrt{\log\left(\frac{2|A||S|^2}{\delta}\right)} $,

we finally get, for any $ \delta \in (0,1) $:

$$
\mathbb{P}\left(\|V^* - \hat{V}\|_\infty \leq \epsilon'\right) \geq 1 - \delta.
$$




## Question 2.3

Suppose once again that we are given a dataset of $N$ samples in the form of tuples $(s_i,a_i,s_i',r_i)$. We know that each tuple contains a valid transition from the true MDP, i.e., $s_i' \sim P(\cdot | s_i, a_i)$ and $r_i \sim R(s_i,a_i)$, while the state-action pairs $(s_i,a_i)$ from which the transition started can be arbitrary.

Suppose we want to apply Q-learning to this MDP. Can you think of a way to leverage this offline data to improve the sample-efficiency of the algorithm? What if we were using SARSA instead?

### **Answer**

We can utilize this dataset by implementing a model-based approach like the Dyna-Q algorithm, which was introduced by Sutton and Barto. The Dyna-Q algorithm combines model-free and model-based learning by using the dataset to simulate additional experiences, thereby improving the sample efficiency.

### **Q-learning with Dyna-Q:**

1. **Direct Learning:** We first use the dataset to update the action-value function $Q$ directly using the standard Q-learning update rule:

    $$
    Q(s, a) \leftarrow Q(s, a) + \alpha_t(s, a) \left(r + \gamma \max_{a'} Q(s', a') - Q(s, a)\right),
    $$

    where $(s, a, r, s')$ are sampled from the offline dataset.

2. **Model Learning:** We can use the dataset to estimate the transition probabilities $\hat{P}(s'|s, a)$ and rewards $\hat{R}(s, a)$. Once the model is learned, we can simulate additional experiences by generating samples $(s', r)$ from the model given any state-action pair $(s, a)$.

3. **Simulated Experience:** These simulated experiences can then be used to perform additional Q-learning updates, which enhances the sample efficiency by allowing the algorithm to learn from both real and simulated data.

### **SARSA with Offline Data:**

In the case of SARSA, which is an on-policy method, the algorithm updates the policy using transitions generated by the current policy. To leverage the offline data:

1. **Initialization:** We initialize the action-value function $$Q(s, a)$ using the offline dataset. This can be done by iterating over the dataset and applying the SARSA update rule:

    $$
    Q(s, a) \leftarrow Q(s, a) + \alpha_t(s, a) \left(r + \gamma Q(s', a') - Q(s, a)\right),
    $$

    where $(s, a, r, s')$ are sampled from the offline dataset, and $$a'$ is the action chosen according to the current policy $\pi$.

2. **Policy Update:** After updating $Q$, we update the policy $\pi$ to be the $\epsilon$-greedy policy derived from the updated $Q$ values.

3. **Simulated Experience (Optional):** Similar to Dyna-Q, we can use the learned model to simulate additional experiences and further update the $Q$ values using the SARSA update rule.

By leveraging the offline dataset in this way, both Q-learning and SARSA can improve their sample efficiency, leading to faster convergence to the optimal policy.


# Part 3 - RL with Function Approximation

## Question 3.1

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is a regularization term and $\lambda > 0$ is the regularization coefficient.


Consider FQI with *linear* function approximation. That is, for a given feature map $\phi : S \rightarrow \mathbb{R}^d$, we consider a parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. Suppose we are applying FQI on a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. Derive the *closed-form* update to find $\theta_{k+1}$, using $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as regularization.

### **Answer**

We consider the following function:

$$
K(\theta) = \frac{1}{2} \sum_{i=1}^N \left(\phi(s_i)^T \theta_{a_i} - y_i^k\right)^2 + \lambda \frac{1}{2} \sum_a \|\theta_a\|_2^2.
$$

The gradient of $K(\theta)$ with respect to $\theta_a$ is:

$$
\nabla_{\theta_a} K(\theta) = \sum_{i: a_i = a} \left(\phi(s_i)^T \theta_a - y_i^k\right)\phi(s_i) + \lambda \theta_a = \left(\sum_{i: a_i = a} \phi(s_i) \phi(s_i)^T + \lambda \mathbb{I}_d\right)\theta_a - \sum_{i: a_i = a} y_i^k \phi(s_i).
$$

Setting $\nabla_{\theta_a} K(\theta) = 0$ gives us:

$$
\theta_a = \left(\sum_{i: a_i = a} \phi(s_i) \phi(s_i)^T + \lambda \mathbb{I}_d\right)^{-1} \left(\sum_{i: a_i = a} y_i^k \phi(s_i)\right).
$$

Thus, the updated parameters at iteration $k+1$ are:

$$
\theta_{k+1} = (\theta_a)_{a \in \mathcal{A}}.
$$



## Question 3.2

The code below creates a larger gridworld (with more states than the one used in the previous questions), and defines a feature map. Implement linear FQI to this environment (in the function `linear_fqi()` below), and compare the approximated $Q$ function to the optimal $Q$ function computed with value iteration.

Can you improve the feature map in order to reduce the approximation error?

### **Answer**

In [ ]:
def get_large_gridworld():
  """Creates an instance of a grid-world MDP with more states."""
  walls = [(ii, 10) for ii in range(15) if (ii != 7 and ii != 8)]
  env = GridWorld(
      nrows=15,
      ncols=15,
      reward_at = {(14, 14):1.0},
      walls=tuple(walls),
      success_probability=0.9,
      terminal_states=((14, 14),)
  )
  return env


class GridWorldFeatureMap:
  """Create features for state-action pairs

  Args:
    dim: int
      Feature dimension
    sigma: float
      RBF kernel bandwidth
  """
  def __init__(self, env, dim=15, sigma=0.25):
    self.index2coord = env.index2coord
    self.n_states = env.Ns
    self.n_actions = env.Na
    self.dim = dim
    self.sigma = sigma

    n_rows = env.nrows
    n_cols = env.ncols

    # build similarity matrix
    sim_matrix = np.zeros((self.n_states, self.n_states))
    for ii in range(self.n_states):
        row_ii, col_ii = self.index2coord[ii]
        x_ii = row_ii / n_rows
        y_ii = col_ii / n_cols
        for jj in range(self.n_states):
            row_jj, col_jj = self.index2coord[jj]
            x_jj = row_jj / n_rows
            y_jj = col_jj / n_cols
            dist = np.sqrt((x_jj - x_ii) ** 2.0 + (y_jj - y_ii) ** 2.0)
            sim_matrix[ii, jj] = np.exp(-(dist / sigma) ** 2.0)

    # factorize similarity matrix to obtain features
    uu, ss, vh = np.linalg.svd(sim_matrix, hermitian=True)
    self.feats = vh[:dim, :]

  def map(self, observation):
    feat = self.feats[:, observation].copy()
    return feat

In [ ]:
env = get_large_gridworld()
feat_map = GridWorldFeatureMap(env)

# Visualize large gridworld
render_policy(env)

# The features have dimension (feature_dim).
feature_example = feat_map.map(1) # feature representation of s=1
print(feature_example)

# Initial vector theta representing the Q function
theta = np.zeros((feat_map.dim, env.action_space.n))
print(theta.shape)
print(feature_example @ theta) # approximation of Q(s=1, a)

In [ ]:
def linear_fqi(env, feat_map, num_iterations, lambd=0.1, gamma=0.95):
  """
  # Linear FQI implementation
  # TO BE COMPLETED
  """
  n_samples=500
  # get a dataset
  dataset = get_uniform_dataset(env, n_samples=n_samples)

  # OR dataset = get_random_policy_dataset(env, n_samples=...)


  theta = np.zeros((feat_map.dim, env.Na))
  Q= np.zeros((env.Ns, env.Na))

  for it in range(num_iterations):

    for a in range(env.Na):
      u=0
      v=0
      index = np.where(np.array(dataset[1]) == 0)[0]
      for i in index:
        u+= np.outer(feat_map.map(dataset[0][i]), (feat_map.map(dataset[0][i])))
        v+= feat_map.map(dataset[0][i])* y_i_

      z=z=inv(u+lambd*np.eye(feat_map.dim)) @ v
      theta[:,a]=z
      for s in range(env.Ns):
        Q[s,a]= np.vdot(feat_map.map(s), theta[:, a])
    pass

  return theta

# ----------------------------
# Environment and feature map
# ----------------------------
env = get_large_gridworld()
# you can change the parameters of the feature map, and even try other maps!
feat_map = GridWorldFeatureMap(env, dim=15, sigma=0.25)

theta = linear_fqi(env, feat_map, num_iterations=1000)

# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy_fqi = Q_fqi.argmax(axis=1)
render_policy(env, policy_fqi, horizon=100)

# Visualize the approximate value function in the gridworld.
img = env.get_layout_img(V_fqi)
plt.imshow(img)
plt.show()